# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time 

import gmaps.datasets

#from ipywidgets.embed import embed_minimal_html 


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../output_data/clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Faanui,-16.48,-151.75,79.23,77,97,11.81,PF,1603589829
1,Las Vigas de Ramírez,19.63,-97.08,49.57,100,49,3.71,MX,1603590003
2,Ilulissat,69.22,-51.10,19.40,57,5,13.87,GL,1603589846
3,Clyde,41.30,-82.98,44.01,73,68,5.35,US,1603590007
4,Rikitea,-23.12,-134.97,74.89,81,99,20.58,PF,1603589724
...,...,...,...,...,...,...,...,...,...
548,Anori,-3.77,-61.64,74.28,98,100,2.15,BR,1603591174
549,Heihe,50.24,127.49,42.80,41,0,15.66,CN,1603591068
550,Lālmohan,22.34,90.74,79.66,81,80,5.82,BD,1603591179
551,Turtas,58.94,69.13,28.33,98,100,10.47,RU,1603591181


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [4]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the heat maps in the images folder to make sure its the required angle.
***

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,Sitrah,26.15,50.62,78.80,74,0,2.24,BH,1603590224
157,Manmād,20.25,74.45,74.46,62,0,3.85,IN,1603590336
247,Salalah,17.02,54.09,71.60,46,0,3.36,OM,1603590512
249,Domoni,-12.26,44.53,76.15,79,0,8.19,KM,1603590532
259,Saint-Philippe,-21.36,55.77,71.60,49,0,6.93,RE,1603590553
284,Māndalgarh,25.20,75.10,70.56,42,0,5.21,IN,1603590607
315,Bandar-e Lengeh,26.56,54.88,76.69,52,0,7.45,IR,1603590674
368,Mandlā,22.60,80.38,71.26,65,0,0.81,IN,1603590789
421,Carutapera,-1.20,-46.02,76.84,86,0,5.84,BR,1603590903
427,Nioro,13.35,-15.75,77.86,90,0,5.57,GM,1603590916


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Issues with finding a hotel at Māndalgarh
Issues with finding a hotel at Nioro


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
105,Sitrah,26.15,50.62,78.80,74,0,2.24,BH,1603590224,Al Bander Resort - منتجع البندر
157,Manmād,20.25,74.45,74.46,62,0,3.85,IN,1603590336,Hotel Visava
247,Salalah,17.02,54.09,71.60,46,0,3.36,OM,1603590512,Atana Stay Salalah
249,Domoni,-12.26,44.53,76.15,79,0,8.19,KM,1603590532,Al Djadid
259,Saint-Philippe,-21.36,55.77,71.60,49,0,6.93,RE,1603590553,Les Embruns Du Baril
284,Māndalgarh,25.20,75.10,70.56,42,0,5.21,IN,1603590607,NA
315,Bandar-e Lengeh,26.56,54.88,76.69,52,0,7.45,IR,1603590674,Diplomat Hotel
368,Mandlā,22.60,80.38,71.26,65,0,0.81,IN,1603590789,Tiger Woods Kanha Resort & Spa (Tiger Sanctuary)
421,Carutapera,-1.20,-46.02,76.84,86,0,5.84,BR,1603590903,HOTEL R. SOARES
427,Nioro,13.35,-15.75,77.86,90,0,5.57,GM,1603590916,NA


In [8]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
105,Sitrah,26.15,50.62,78.80,74,0,2.24,BH,1603590224,Al Bander Resort - منتجع البندر
157,Manmād,20.25,74.45,74.46,62,0,3.85,IN,1603590336,Hotel Visava
247,Salalah,17.02,54.09,71.60,46,0,3.36,OM,1603590512,Atana Stay Salalah
249,Domoni,-12.26,44.53,76.15,79,0,8.19,KM,1603590532,Al Djadid
259,Saint-Philippe,-21.36,55.77,71.60,49,0,6.93,RE,1603590553,Les Embruns Du Baril
315,Bandar-e Lengeh,26.56,54.88,76.69,52,0,7.45,IR,1603590674,Diplomat Hotel
368,Mandlā,22.60,80.38,71.26,65,0,0.81,IN,1603590789,Tiger Woods Kanha Resort & Spa (Tiger Sanctuary)
421,Carutapera,-1.20,-46.02,76.84,86,0,5.84,BR,1603590903,HOTEL R. SOARES
451,Bandrele,-12.91,45.19,78.80,78,0,4.70,YT,1603590783,Hôtel Restaurant Sakouli
481,Bonito,-21.12,-56.48,75.60,78,0,1.36,BR,1603591031,Zagaia Eco Resort


In [9]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

11

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Checking hotel info:
len(hotel_info)

11

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the hotel maps in the images folder to make sure its the required angle. 
***